In [ ]:
# This file contains tests for CharismaMixin
import os
import sys
from datetime import date
import shutil
import warnings
warnings.filterwarnings('ignore')

import numpy as np

sys.path.append('../..') # for running py-script
sys.path.append('../../..') # for running this notebook directly

from seismiqb import Field
from seismiqb.src.geometry import export

In [ ]:
""" The behaviour of tests is parametrized by the following constants:

DATESTAMP : str
    Execution date in "YYYY-MM-DD" format.
    Used for saving notebooks executions and temporary files.
USE_TMP_OUTPUT_DIR: bool
    Whether to use pytest tmpdir as a workspace.
    If True, then all files are saved in temporary directories.
    If False, then all files are saved in local directories.
OUTPUT_DIR : str
    Path to the directory for saving results and temporary files
    (executed notebooks, logs, data files like cubes, etc.).

And you can manage tests running with parameters:

REMOVE_OUTDATED_FILES: bool
    Whether to remove outdated files which relate to previous executions.
REMOVE_EXTRA_FILES : bool
    Whether to remove extra files after execution.
    Extra files are temporary files and execution savings that relate to successful tests.

You can manage parameters which relates to a cube generation:

CUBE_SHAPE : sequence of three integers
    Shape of a synthetic cube.
SEED: int or None
    Seed used for creation of random generator (check out `np.random.default_rng`).
"""

# Defaults for run directly this notebook
# Workspace constants
DATESTAMP = date.today().strftime("%Y-%m-%d")
USE_TMP_OUTPUT_DIR = False
TESTS_ROOT_DIR = './'

# Execution parameters
REMOVE_EXTRA_FILES = True

# Data creation parameters
CUBE_SHAPE = (100, 100, 100)
SEED = 10

# Preparation

**Storage structure:**
___



**charisma_test_files** (tests root directory with temporary files)

&emsp;├── **test_array.npy** (Test cube data, now it is a random noise)

&emsp;├── **test_cube.sgy** (Test cube data in the sgy format)

&emsp;├── **test_cube.meta** (Meta data of the test cube)
    
&emsp;├── **test_matrix** (Saved test matrix)

&emsp;└── **test_points** (Saved test points)


In [ ]:
# (Re)create the test directory
OUTPUT_DIR = os.path.join(TESTS_ROOT_DIR, f"charisma_test_files")

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
%%time
# Create a fake cube
CUBE_PATH = os.path.join(OUTPUT_DIR, f"test_cube_{DATESTAMP}.sgy")

rng = np.random.default_rng(SEED)
data_array = rng.normal(0, 1000, CUBE_SHAPE).astype(np.float32)

with open(os.path.join(OUTPUT_DIR, f"test_array_{DATESTAMP}.npy"), "wb") as outfile:
    np.save(outfile, data_array)

export.make_segy_from_array(array=data_array, path_segy=CUBE_PATH, zip_segy=False,
                            sample_rate=2., delay=50, pbar="t")

In [ ]:
%%time
# Init a field
field = Field(geometry=CUBE_PATH)

# Tests

In [ ]:
%%time
# Test dump_charisma, load_charisma and is_charisma_like (matrix case)
# Create and dump a random matrix
matrix = rng.integers(low=0, high=10, size=CUBE_SHAPE[:2])

path=os.path.join(OUTPUT_DIR, f"test_matrix_{DATESTAMP}")

field.dump_charisma(data=matrix, path=path, format="matrix", name="saved_matrix")

# Check is_charisma_like
assert field.is_charisma_like(path), "A dumped matrix file is not charisma-like."

# Open saved data
opened_matrix = field.load_charisma(path=path, format="matrix")

assert np.array_equal(matrix, opened_matrix), "Saved and loaded matrices are not the same."

In [ ]:
%%time
# Test dump_charisma, load_charisma and is_charisma_like (points case)
# Create random points in the field
i_lines_range = np.linspace(0, CUBE_SHAPE[0]-1, CUBE_SHAPE[0]).astype(int)
x_lines_range = np.linspace(0, CUBE_SHAPE[1]-1, CUBE_SHAPE[1]).astype(int)

i_lines, x_lines = np.meshgrid(i_lines_range, x_lines_range)

depths = rng.integers(low=0, high=10, size=(i_lines.size, 1))

points = np.hstack([i_lines.reshape(i_lines.size, 1), x_lines.reshape(x_lines.size, 1), depths])

# Sort for dumped and loaded points consistency
points = np.sort(points, axis=1)
points = np.sort(points, axis=0)

# Save points as charisma
path=os.path.join(OUTPUT_DIR, f"test_points_{DATESTAMP}")

field.dump_charisma(data=points, path=path, format="points", name="saved_points")

# Check is_charisma_like
assert field.is_charisma_like(path), "A dumped points file is not charisma-like."

# Open saved data
opened_points = field.load_charisma(path=path, format="points")

assert np.array_equal(points, opened_points), "Saved and loaded points arrays are not the same."

# Exit

In [ ]:
if REMOVE_EXTRA_FILES:
    shutil.rmtree(OUTPUT_DIR)